In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import plotly as py
import cufflinks as cf
cf.go_offline()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math
from IPython.display import display, HTML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate

In [2]:
cannabis = pd.read_csv('../data/cannabis.csv')
cannabis = cannabis[cannabis["Rating"] >=4]

cannabis.to_csv(r'../data/cannabis2.csv')
cannabis.shape

(2162, 6)

In [3]:
cannabis.head()
data = pd.read_csv('../data/cannabis2.csv')
data.head()
data_effect = pd.DataFrame(data.Effects.str.split(',',4).tolist(),
             columns = ['Effect_1','Effect_2','Effect_3','Effect_4','Effect_5'])

data_flavors = pd.DataFrame(data.Flavor.str.split(',',n=2,expand=True).values.tolist(),
                          columns = ['Flavor_1','Flavor_2','Flavor_3'])
data = pd.concat([data, data_effect], axis=1)
data = pd.concat([data, data_flavors], axis=1)
del data["Effects"]
del data["Flavor"]
del data["Unnamed: 0"]
data.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


,Unnamed: 0,Strain,Type,Rating,Effects,Flavor,Description
0,0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


,Strain,Type,Rating,Description,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5,Flavor_1,Flavor_2,Flavor_3
0,100-Og,hybrid,4.0,$100 OG is a 50/50 hybrid strain that packs a ...,Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus
1,98-White-Widow,hybrid,4.7,The ‘98 Aloha White Widow is an especially pot...,Relaxed,Aroused,Creative,Happy,Energetic,Flowery,Violet,Diesel
2,1024,sativa,4.4,1024 is a sativa-dominant hybrid bred in Spain...,Uplifted,Happy,Relaxed,Energetic,Creative,Spicy/Herbal,Sage,Woody
3,13-Dawgs,hybrid,4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly,Creative,Hungry,Relaxed,Uplifted,Apricot,Citrus,Grapefruit
4,24K-Gold,hybrid,4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy,Relaxed,Euphoric,Uplifted,Talkative,Citrus,Earthy,Orange


In [4]:
dummy = pd.get_dummies(data[['Effect_1','Effect_2','Effect_3','Effect_4','Effect_5','Flavor_1','Flavor_2','Flavor_3']])
data = pd.concat([data, dummy], axis=1)

In [5]:
data = data.drop(['Effect_1','Effect_2','Effect_3','Effect_4',
                        'Effect_5','Flavor_1','Flavor_2','Flavor_3',"Description"], axis=1)
data.head()

,Strain,Type,Rating,Effect_1_Aroused,Effect_1_Creative,Effect_1_Dry,Effect_1_Energetic,Effect_1_Euphoric,Effect_1_Focused,Effect_1_Giggly,Effect_1_Happy,Effect_1_Hungry,Effect_1_None,Effect_1_Relaxed,Effect_1_Sleepy,Effect_1_Talkative,Effect_1_Tingly,Effect_1_Uplifted,Effect_2_Aroused,Effect_2_Creative,Effect_2_Energetic,Effect_2_Euphoric,Effect_2_Focused,Effect_2_Giggly,Effect_2_Happy,Effect_2_Hungry,Effect_2_Mouth,Effect_2_Relaxed,Effect_2_Sleepy,Effect_2_Talkative,Effect_2_Tingly,Effect_2_Uplifted,Effect_3_Aroused,Effect_3_Creative,Effect_3_Energetic,Effect_3_Euphoric,Effect_3_Focused,Effect_3_Giggly,Effect_3_Happy,Effect_3_Hungry,Effect_3_Relaxed,Effect_3_Sleepy,Effect_3_Talkative,Effect_3_Tingly,Effect_3_Uplifted,Effect_4_Aroused,Effect_4_Creative,Effect_4_Energetic,Effect_4_Euphoric,Effect_4_Focused,Effect_4_Giggly,Effect_4_Happy,Effect_4_Hungry,Effect_4_Relaxed,Effect_4_Sleepy,Effect_4_Talkative,Effect_4_Tingly,Effect_4_Uplifted,Effect_5_Aroused,Effect_5_Creative,Effect_5_Energetic,Effect_5_Euphoric,Effect_5_Focused,Effect_5_Giggly,Effect_5_Happy,Effect_5_Hungry,Effect_5_Relaxed,Effect_5_Sleepy,Effect_5_Talkative,Effect_5_Tingly,Effect_5_Uplifted,Flavor_1_Ammonia,Flavor_1_Apple,Flavor_1_Apricot,Flavor_1_Berry,Flavor_1_Blue,Flavor_1_Blueberry,Flavor_1_Butter,Flavor_1_Cheese,Flavor_1_Chemical,Flavor_1_Citrus,Flavor_1_Coffee,Flavor_1_Diesel,Flavor_1_Earthy,Flavor_1_Flowery,Flavor_1_Grape,Flavor_1_Grapefruit,Flavor_1_Honey,Flavor_1_Lavender,Flavor_1_Lemon,Flavor_1_Lime,Flavor_1_Mango,Flavor_1_Menthol,Flavor_1_Mint,Flavor_1_Minty,Flavor_1_None,Flavor_1_Nutty,Flavor_1_Orange,Flavor_1_Peach,Flavor_1_Pear,Flavor_1_Pepper,Flavor_1_Pine,Flavor_1_Pineapple,Flavor_1_Plum,Flavor_1_Pungent,Flavor_1_Rose,Flavor_1_Sage,Flavor_1_Skunk,Flavor_1_Spicy/Herbal,Flavor_1_Strawberry,Flavor_1_Sweet,Flavor_1_Tar,Flavor_1_Tea,Flavor_1_Tobacco,Flavor_1_Tree,Flavor_1_Tropical,Flavor_1_Vanilla,Flavor_1_Violet,Flavor_1_Woody,Flavor_2_Ammonia,Flavor_2_Apple,Flavor_2_Apricot,Flavor_2_Berry,Flavor_2_Blue,Flavor_2_Blueberry,Flavor_2_Butter,Flavor_2_Cheese,Flavor_2_Chemical,Flavor_2_Chestnut,Flavor_2_Citrus,Flavor_2_Coffee,Flavor_2_Diesel,Flavor_2_Earthy,Flavor_2_Flowery,Flavor_2_Fruit,Flavor_2_Grape,Flavor_2_Grapefruit,Flavor_2_Honey,Flavor_2_Lavender,Flavor_2_Lemon,Flavor_2_Lime,Flavor_2_Mango,Flavor_2_Menthol,Flavor_2_Mint,Flavor_2_Minty,Flavor_2_Nutty,Flavor_2_Orange,Flavor_2_Pear,Flavor_2_Pepper,Flavor_2_Pine,Flavor_2_Pineapple,Flavor_2_Plum,Flavor_2_Pungent,Flavor_2_Rose,Flavor_2_Sage,Flavor_2_Skunk,Flavor_2_Spicy/Herbal,Flavor_2_Strawberry,Flavor_2_Sweet,Flavor_2_Tea,Flavor_2_Tobacco,Flavor_2_Tree,Flavor_2_Tropical,Flavor_2_Vanilla,Flavor_2_Violet,Flavor_2_Woody,Flavor_3_Ammonia,Flavor_3_Apple,Flavor_3_Apricot,Flavor_3_Berry,"Flavor_3_Blue,Cheese",Flavor_3_Blueberry,Flavor_3_Butter,Flavor_3_Cheese,"Flavor_3_Cheese,Blueberry","Flavor_3_Cheese,Chemical","Flavor_3_Cheese,Earthy",Flavor_3_Chemical,Flavor_3_Chestnut,Flavor_3_Citrus,Flavor_3_Coffee,Flavor_3_Diesel,Flavor_3_Earthy,"Flavor_3_Earthy,Pine","Flavor_3_Earthy,Sweet",Flavor_3_Flowery,"Flavor_3_Fruit,Apricot","Flavor_3_Fruit,Berry","Flavor_3_Fruit,Earthy","Flavor_3_Fruit,Flowery","Flavor_3_Fruit,Lemon","Flavor_3_Fruit,Pine","Flavor_3_Fruit,Pineapple","Flavor_3_Fruit,Spicy/Herbal","Flavor_3_Fruit,Sweet",Flavor_3_Grape,Flavor_3_Grapefruit,Flavor_3_Honey,Flavor_3_Lavender,Flavor_3_Lemon,Flavor_3_Lime,Flavor_3_Mango,Flavor_3_Menthol,"Flavor_3_Menthol,Flowery",Flavor_3_Mint,Flavor_3_Minty,Flavor_3_Nutty,Flavor_3_Orange,Flavor_3_Peach,Flavor_3_Pear,Flavor_3_Pepper,Flavor_3_Pine,Flavor_3_Pineapple,Flavor_3_Pungent,"Flavor_3_Pungent,Pine",Flavor_3_Rose,Flavor_3_Sage,Flavor_3_Skunk,Flavor_3_Spicy/Herbal,Flavor_3_Strawberry,Flavor_3_Sweet,"Flavor_3_Sweet,Grape",Flavor_3_Tar,Flavor_3_Tea,Flavor_3_Tobacco,"Flavor_3_Tree,Fruit",Flavor_3_Tropical,Flavor_3_Vanilla,Flavor_3_Violet,Flavor_3_Woody
0,100-Og,hybrid,4.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,

In [6]:
del data["Type"]
del data["Rating"]

In [7]:
d1 =data.set_index('Strain')

In [8]:
d1.T.head()
d1.shape


Strain              100-Og  98-White-Widow  1024  13-Dawgs  24K-Gold  3-Kings  \
Effect_1_Aroused         0               0     0         0         0        0   
Effect_1_Creative        1               0     0         0         0        0   
Effect_1_Dry             0               0     0         0         0        0   
Effect_1_Energetic       0               0     0         0         0        0   
Effect_1_Euphoric        0               0     0         0         0        0   

Strain              303-Og  3D-Cbd  3X-Crazy  3Rd-Coast-Panama-Chunk  \
Effect_1_Aroused         0       0         0                       0   
Effect_1_Creative        0       0         0                       0   
Effect_1_Dry             0       0         0                       0   
Effect_1_Energetic       0       0         0                       1   
Effect_1_Euphoric        0       0         0                       0   

Strain              501St-Og  5Th-Element  707-Headband  8-Ball-Kush  818-Og  \
Effect_1_Aroused           0            0             0            0       0   
Effect_1_Creative          0            0             0            0       0   
Effect_1_Dry               0            0             0            0       0   
Effect_1_Energetic         0            0             0            0       0   
Effect_1_Euphoric          0            0             1            0       0   

Strain              831-Og  9-Pound-Hammer  91-Krypt  A-Dub  A-Train  Acdc  \
Effect_1_Aroused         0               0         0      0        0     0   
Effect_1_Creative        0               0         0      0        1     0   
Effect_1_Dry             0               0         0      0        0     0   
Effect_1_Energetic       0               0         0      0        0     0   
Effect_1_Euphoric        0               0         0      0        0     0   

Strain              Ak-47  Ak-48  Aberdeen  Abusive-Og  Acapulco-Gold  \
Effect_1_Aroused        0      0         0           0              0   
Effect_1_Creative       0      0         0           0              0   
Effect_1_Dry            0      0         0           0              0   
Effect_1_Energetic      0      0         0           0              0   
Effect_1_Euphoric       0      0         0           0              0   

Strain              Ace-Killer-Og  Ace-Of-Spades  Aceh  Acid  Acid-Dough  \
Effect_1_Aroused                0              0     0     0           0   
Effect_1_Creative               0              0     1     1           0   
Effect_1_Dry                    0              0     0     0           0   
Effect_1_Energetic              0              0     0     0           0   
Effect_1_Euphoric               0              0     0     0           0   

Strain              Afghan-Big-Bud  Afghan-Cow  Afghan-Hawaiian  Afghan-Haze  \
Effect_1_Aroused                 0           0                0            0   
Effect_1_Creative                0           0                0            0   
Effect_1_Dry                     0           0                0            0   
Effect_1_Energetic               0           0                0            0   
Effect_1_Euphoric                1           0                0            0   

Strain              Afghan-Kush  Afghan-Skunk  Afghan-Sour-Kush  Afghan-Widow  \
Effect_1_Aroused              0             0                 0             0   
Effect_1_Creative             0             0                 0             1   
Effect_1_Dry                  0             0                 0             0   
Effect_1_Energetic            0             0                 0             0   
Effect_1_Euphoric             0             0                 0             0   

Strain              Afghani  Afgahni-Bullrider  Afghani-Cbd  Afghanica  \
Effect_1_Aroused          0                  0            0          0   
Effect_1_Creative         0                  0            0          0   
Effect_1_Dry              0                 

(2162, 227)

In [9]:
d1.loc['prueba'] = 0
d1.loc['prueba'] = {'Effect_1_Aroused':1,'Effect_2_Creative':1,'Flavor_2_Citrus':1}
d1 = d1.fillna(0)
d1.tail()
d2 = d1.T
d2.tail()

,Effect_1_Aroused,Effect_1_Creative,Effect_1_Dry,Effect_1_Energetic,Effect_1_Euphoric,Effect_1_Focused,Effect_1_Giggly,Effect_1_Happy,Effect_1_Hungry,Effect_1_None,Effect_1_Relaxed,Effect_1_Sleepy,Effect_1_Talkative,Effect_1_Tingly,Effect_1_Uplifted,Effect_2_Aroused,Effect_2_Creative,Effect_2_Energetic,Effect_2_Euphoric,Effect_2_Focused,Effect_2_Giggly,Effect_2_Happy,Effect_2_Hungry,Effect_2_Mouth,Effect_2_Relaxed,Effect_2_Sleepy,Effect_2_Talkative,Effect_2_Tingly,Effect_2_Uplifted,Effect_3_Aroused,Effect_3_Creative,Effect_3_Energetic,Effect_3_Euphoric,Effect_3_Focused,Effect_3_Giggly,Effect_3_Happy,Effect_3_Hungry,Effect_3_Relaxed,Effect_3_Sleepy,Effect_3_Talkative,Effect_3_Tingly,Effect_3_Uplifted,Effect_4_Aroused,Effect_4_Creative,Effect_4_Energetic,Effect_4_Euphoric,Effect_4_Focused,Effect_4_Giggly,Effect_4_Happy,Effect_4_Hungry,Effect_4_Relaxed,Effect_4_Sleepy,Effect_4_Talkative,Effect_4_Tingly,Effect_4_Uplifted,Effect_5_Aroused,Effect_5_Creative,Effect_5_Energetic,Effect_5_Euphoric,Effect_5_Focused,Effect_5_Giggly,Effect_5_Happy,Effect_5_Hungry,Effect_5_Relaxed,Effect_5_Sleepy,Effect_5_Talkative,Effect_5_Tingly,Effect_5_Uplifted,Flavor_1_Ammonia,Flavor_1_Apple,Flavor_1_Apricot,Flavor_1_Berry,Flavor_1_Blue,Flavor_1_Blueberry,Flavor_1_Butter,Flavor_1_Cheese,Flavor_1_Chemical,Flavor_1_Citrus,Flavor_1_Coffee,Flavor_1_Diesel,Flavor_1_Earthy,Flavor_1_Flowery,Flavor_1_Grape,Flavor_1_Grapefruit,Flavor_1_Honey,Flavor_1_Lavender,Flavor_1_Lemon,Flavor_1_Lime,Flavor_1_Mango,Flavor_1_Menthol,Flavor_1_Mint,Flavor_1_Minty,Flavor_1_None,Flavor_1_Nutty,Flavor_1_Orange,Flavor_1_Peach,Flavor_1_Pear,Flavor_1_Pepper,Flavor_1_Pine,Flavor_1_Pineapple,Flavor_1_Plum,Flavor_1_Pungent,Flavor_1_Rose,Flavor_1_Sage,Flavor_1_Skunk,Flavor_1_Spicy/Herbal,Flavor_1_Strawberry,Flavor_1_Sweet,Flavor_1_Tar,Flavor_1_Tea,Flavor_1_Tobacco,Flavor_1_Tree,Flavor_1_Tropical,Flavor_1_Vanilla,Flavor_1_Violet,Flavor_1_Woody,Flavor_2_Ammonia,Flavor_2_Apple,Flavor_2_Apricot,Flavor_2_Berry,Flavor_2_Blue,Flavor_2_Blueberry,Flavor_2_Butter,Flavor_2_Cheese,Flavor_2_Chemical,Flavor_2_Chestnut,Flavor_2_Citrus,Flavor_2_Coffee,Flavor_2_Diesel,Flavor_2_Earthy,Flavor_2_Flowery,Flavor_2_Fruit,Flavor_2_Grape,Flavor_2_Grapefruit,Flavor_2_Honey,Flavor_2_Lavender,Flavor_2_Lemon,Flavor_2_Lime,Flavor_2_Mango,Flavor_2_Menthol,Flavor_2_Mint,Flavor_2_Minty,Flavor_2_Nutty,Flavor_2_Orange,Flavor_2_Pear,Flavor_2_Pepper,Flavor_2_Pine,Flavor_2_Pineapple,Flavor_2_Plum,Flavor_2_Pungent,Flavor_2_Rose,Flavor_2_Sage,Flavor_2_Skunk,Flavor_2_Spicy/Herbal,Flavor_2_Strawberry,Flavor_2_Sweet,Flavor_2_Tea,Flavor_2_Tobacco,Flavor_2_Tree,Flavor_2_Tropical,Flavor_2_Vanilla,Flavor_2_Violet,Flavor_2_Woody,Flavor_3_Ammonia,Flavor_3_Apple,Flavor_3_Apricot,Flavor_3_Berry,"Flavor_3_Blue,Cheese",Flavor_3_Blueberry,Flavor_3_Butter,Flavor_3_Cheese,"Flavor_3_Cheese,Blueberry","Flavor_3_Cheese,Chemical","Flavor_3_Cheese,Earthy",Flavor_3_Chemical,Flavor_3_Chestnut,Flavor_3_Citrus,Flavor_3_Coffee,Flavor_3_Diesel,Flavor_3_Earthy,"Flavor_3_Earthy,Pine","Flavor_3_Earthy,Sweet",Flavor_3_Flowery,"Flavor_3_Fruit,Apricot","Flavor_3_Fruit,Berry","Flavor_3_Fruit,Earthy","Flavor_3_Fruit,Flowery","Flavor_3_Fruit,Lemon","Flavor_3_Fruit,Pine","Flavor_3_Fruit,Pineapple","Flavor_3_Fruit,Spicy/Herbal","Flavor_3_Fruit,Sweet",Flavor_3_Grape,Flavor_3_Grapefruit,Flavor_3_Honey,Flavor_3_Lavender,Flavor_3_Lemon,Flavor_3_Lime,Flavor_3_Mango,Flavor_3_Menthol,"Flavor_3_Menthol,Flowery",Flavor_3_Mint,Flavor_3_Minty,Flavor_3_Nutty,Flavor_3_Orange,Flavor_3_Peach,Flavor_3_Pear,Flavor_3_Pepper,Flavor_3_Pine,Flavor_3_Pineapple,Flavor_3_Pungent,"Flavor_3_Pungent,Pine",Flavor_3_Rose,Flavor_3_Sage,Flavor_3_Skunk,Flavor_3_Spicy/Herbal,Flavor_3_Strawberry,Flavor_3_Sweet,"Flavor_3_Sweet,Grape",Flavor_3_Tar,Flavor_3_Tea,Flavor_3_Tobacco,"Flavor_3_Tree,Fruit",Flavor_3_Tropical,Flavor_3_Vanilla,Flavor_3_Violet,Flavor_3_Woody
Strain,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Strain               100-Og  98-White-Widow  1024  13-Dawgs  24K-Gold  \
Flavor_3_Tree,Fruit     0.0             0.0   0.0       0.0       0.0   
Flavor_3_Tropical       0.0             0.0   0.0       0.0       0.0   
Flavor_3_Vanilla        0.0             0.0   0.0       0.0       0.0   
Flavor_3_Violet         0.0             0.0   0.0       0.0       0.0   
Flavor_3_Woody          0.0             0.0   1.0       0.0       0.0   

Strain               3-Kings  303-Og  3D-Cbd  3X-Crazy  \
Flavor_3_Tree,Fruit      0.0     0.0     0.0       0.0   
Flavor_3_Tropical        0.0     0.0     0.0       0.0   
Flavor_3_Vanilla         0.0     0.0     0.0       0.0   
Flavor_3_Violet          0.0     0.0     0.0       0.0   
Flavor_3_Woody           0.0     0.0     0.0       0.0   

Strain               3Rd-Coast-Panama-Chunk  501St-Og  5Th-Element  \
Flavor_3_Tree,Fruit                     0.0       0.0          0.0   
Flavor_3_Tropical                       0.0       0.0          0.0   
Flavor_3_Vanilla                        0.0       0.0          0.0   
Flavor_3_Violet                         0.0       0.0          0.0   
Flavor_3_Woody                          0.0       0.0          0.0   

Strain               707-Headband  8-Ball-Kush  818-Og  831-Og  \
Flavor_3_Tree,Fruit           0.0          0.0     0.0     0.0   
Flavor_3_Tropical             0.0          0.0     0.0     0.0   
Flavor_3_Vanilla              0.0          0.0     0.0     0.0   
Flavor_3_Violet               0.0          0.0     0.0     0.0   
Flavor_3_Woody                0.0          0.0     0.0     0.0   

Strain               9-Pound-Hammer  91-Krypt  A-Dub  A-Train  Acdc  Ak-47  \
Flavor_3_Tree,Fruit             0.0       0.0    0.0      0.0   0.0    0.0   
Flavor_3_Tropical               0.0       0.0    0.0      0.0   0.0    0.0   
Flavor_3_Vanilla                0.0       0.0    0.0      0.0   0.0    0.0   
Flavor_3_Violet                 0.0       0.0    0.0      0.0   0.0    0.0   
Flavor_3_Woody                  0.0       0.0    0.0      0.0   1.0    1.0   

Strain               Ak-48  Aberdeen  Abusive-Og  Acapulco-Gold  \
Flavor_3_Tree,Fruit    0.0       0.0         0.0            0.0   
Flavor_3_Tropical      0.0       0.0         0.0            0.0   
Flavor_3_Vanilla       0.0       0.0         0.0            0.0   
Flavor_3_Violet        0.0       0.0         0.0            0.0   
Flavor_3_Woody         0.0       0.0         0.0            0.0   

Strain               Ace-Killer-Og  Ace-Of-Spades  Aceh  Acid  Acid-Dough  \
Flavor_3_Tree,Fruit            0.0            0.0   0.0   0.0         0.0   
Flavor_3_Tropical              0.0            0.0   0.0   0.0         0.0   
Flavor_3_Vanilla               0.0            0.0   0.0   0.0         0.0   
Flavor_3_Violet                0.0            0.0   0.0   0.0         0.0   
Flavor_3_Woody                 0.0            1.0   0.0   0.0         0.0   

Strain               Afghan-Big-Bud  Afghan-Cow  Afghan-Hawaiian  Afghan-Haze  \
Flavor_3_Tree,Fruit             0.0         0.0              0.0          0.0   
Flavor_3_Tropical               0.0         0.0              0.0          0.0   
Flavor_3_Vanilla                0.0         0.0              0.0          0.0   
Flavor_3_Violet                 0.0         0.0              0.0          0.0   
Flavor_3_Woody                  0.0         0.0              0.0          0.0   

Strain               Afghan-Kush  Afghan-Skunk  Afghan-Sour-Kush  \
Flavor_3_Tree,Fruit          0.0           0.0               0.0   
Flavor_3_Tropical            0.0           0.0               0.0   
Flavor_3_Vanilla             0.0           0.0               0.0   
Flavor_3_Violet              0.0           0.0               0.0   
Flavor_3_Woody               1.0           0.0               0.0   

Strain               Afghan-Widow  Afghani  Afgahni-Bullrider  Afghani-Cbd  \
Flavor_3_Tree,Fruit           0.0      0.0                0.0          0.0   
Flavor_

In [10]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(d2.T, 'cosine'))

array([[0.   , 1.   , 1.   , ..., 1.   , 1.   , 1.   ],
       [1.   , 0.   , 1.   , ..., 0.75 , 0.875, 1.   ],
       [1.   , 1.   , 0.   , ..., 1.   , 1.   , 1.   ],
       ...,
       [1.   , 0.75 , 1.   , ..., 0.   , 0.625, 1.   ],
       [1.   , 0.875, 1.   , ..., 0.625, 0.   , 1.   ],
       [1.   , 1.   , 1.   , ..., 1.   , 1.   , 0.   ]])

In [11]:
from scipy.spatial.distance import pdist, squareform
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

distances = pd.DataFrame(1/(1 + squareform(pdist(d2.T, 'cosine'))), 
                         index=d2.columns, columns=d2.columns)

distances.head()


Strain          100-Og  98-White-Widow  1024  13-Dawgs  24K-Gold   3-Kings  \
Strain                                                                       
100-Og             1.0             0.5   0.5       0.5       0.5  0.571429   
98-White-Widow     0.5             1.0   0.5       0.5       0.5  0.533333   
1024               0.5             0.5   1.0       0.5       0.5  0.500000   
13-Dawgs           0.5             0.5   0.5       1.0       0.5  0.500000   
24K-Gold           0.5             0.5   0.5       0.5       1.0  0.533333   

Strain            303-Og    3D-Cbd  3X-Crazy  3Rd-Coast-Panama-Chunk  \
Strain                                                                 
100-Og          0.500000  0.571429  0.571429                0.500000   
98-White-Widow  0.533333  0.500000  0.533333                0.500000   
1024            0.533333  0.533333  0.500000                0.538891   
13-Dawgs        0.500000  0.500000  0.533333                0.500000   
24K-Gold        0.615385  0.500000  0.500000                0.538891   

Strain          501St-Og  5Th-Element  707-Headband  8-Ball-Kush    818-Og  \
Strain                                                                       
100-Og          0.533333     0.533333      0.533333     0.500000  0.533333   
98-White-Widow  0.571429     0.533333      0.500000     0.533333  0.533333   
1024            0.500000     0.500000      0.571429     0.500000  0.533333   
13-Dawgs        0.500000     0.500000      0.500000     0.500000  0.500000   
24K-Gold        0.533333     0.571429      0.500000     0.571429  0.533333   

Strain          831-Og  9-Pound-Hammer  91-Krypt     A-Dub   A-Train  \
Strain                                                                 
100-Og             0.5        0.571429  0.533333  0.500000  0.615385   
98-White-Widow     0.5        0.571429  0.533333  0.533333  0.533333   
1024               0.5        0.500000  0.500000  0.533333  0.533333   
13-Dawgs           0.5        0.500000  0.533333  0.500000  0.500000   
24K-Gold           0.5        0.533333  0.500000  0.571429  0.500000   

Strain              Acdc     Ak-47     Ak-48  Aberdeen  Abusive-Og  \
Strain                                                               
100-Og          0.533333  0.571429  0.666667  0.500000    0.533333   
98-White-Widow  0.533333  0.533333  0.500000  0.500000    0.615385   
1024            0.571429  0.533333  0.500000  0.571429    0.500000   
13-Dawgs        0.500000  0.500000  0.500000  0.500000    0.500000   
24K-Gold        0.500000  0.571429  0.533333  0.500000    0.500000   

Strain          Acapulco-Gold  Ace-Killer-Og  Ace-Of-Spades      Aceh  \
Strain                                                                  
100-Og               0.615385       0.533333       0.571429  0.571429   
98-White-Widow       0.500000       0.571429       0.533333  0.500000   
1024                 0.533333       0.500000       0.533333  0.533333   
13-Dawgs             0.500000       0.533333       0.500000  0.500000   
24K-Gold             0.571429       0.533333       0.500000  0.500000   

Strain              Acid  Acid-Dough  Afghan-Big-Bud  Afghan-Cow  \
Strain                                                             
100-Og          0.533333    0.538891        0.500000      0.5358   
98-White-Widow  0.500000    0.500000        0.500000      0.5000   
1024            0.571429    0.500000        0.571429      0.5000   
13-Dawgs        0.533333    0.500000        0.500000      0.5000   
24K-Gold        0.533333    0.500000        0.533333      0.5358   

Strain          Afghan-Hawaiian  Afghan-Haze  Afghan-Kush  Afghan-Skunk  \
Strain                                                                    
100-Og                 0.500000     0.533333     0.533333      0.500000   
98-White-Widow         0.500000     0.533333     0.533333      0.500000   
1024                   0.571429     0.533333     0.533333      0.500000   
13-Dawgs               0.533333     0.500

In [12]:
top5 = distances['prueba'].sort_values(ascending=False)[1:6]
top5

Strain
Full-Moon           0.720654
Euphoria-Cookies    0.628239
Outer-Space         0.628239
Qrazy-Train         0.628239
Jahwaiian           0.628239
Name: prueba, dtype: float64

In [13]:
l = distances.columns
print(l)

Index(['100-Og', '98-White-Widow', '1024', '13-Dawgs', '24K-Gold', '3-Kings',
       '303-Og', '3D-Cbd', '3X-Crazy', '3Rd-Coast-Panama-Chunk',
       ...
       'Zamaldelica', 'Zellys-Gift', 'Zen', 'Zeta-Sage', 'Zeus-Og', 'Zkittlez',
       'Zombie-Kush', 'Zombie-Og', 'Zoom-Pie', 'prueba'],
      dtype='object', name='Strain', length=2163)


In [14]:
prueba4= cannabis[cannabis["Strain"] == 'Full-Moon']
prueba4

,Strain,Type,Rating,Effects,Flavor,Description
827,Full-Moon,sativa,4.9,"Aroused,Creative,Euphoric,Happy,Uplifted","Spicy/Herbal,Citrus,Earthy",Nirvana Seeds found this beauty on the Thai is...


In [15]:
prueba5= cannabis[cannabis["Strain"] == 'Euphoria-Cookies']
prueba5

,Strain,Type,Rating,Effects,Flavor,Description
774,Euphoria-Cookies,hybrid,5.0,"Euphoric,Creative,Uplifted,Happy,Relaxed","Sweet,Citrus,Tropical",Euphoria Cookies is a balanced 50/50 hybrid cr...
